In [1]:
pip install tkintertable

     ---------------------------------------- 0.0/58.7 kB ? eta -:--:--
     ------ --------------------------------- 10.2/58.7 kB ? eta -:--:--
     -------------------------- ----------- 41.0/58.7 kB 388.9 kB/s eta 0:00:01
     -------------------------------------- 58.7/58.7 kB 388.0 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/491.3 kB ? eta -:--:--
   ----- ---------------------------------- 61.4/491.3 kB 3.2 MB/s eta 0:00:01
   ----------------------- ---------------- 286.7/491.3 kB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 491.3/491.3 kB 4.4 MB/s eta 0:00:00
  Created wheel for tkintertable: filename=tkintertable-1.3.3-py3-none-any.whl size=66206 sha256=8763d709ec30e53cec1f331aaf2497f4eb52920b1c99d428479a1e8759a216c6
  Stored in directory: c:\users\ayan\appdata\local\pip\cache\wheels\93\b5\2d\e0c1d9fad21eeab6171702bc918627821cb2c46aecc7

## V1

In [9]:
import tkinter as tk
import math
import random

# ==========================
# Physics constants
# ==========================
G = 0.5    # Gravitational constant
DT = 0.2   # Time step

TRAIL_LENGTH = 150  # max trail points per body
BODY_RADIUS = 12    # fixed visual size

# ==========================
# Body Class
# ==========================
class Body:
    def __init__(self, canvas, x, y, vx, vy, mass, color):
        self.canvas = canvas
        self.x, self.y = x, y
        self.vx, self.vy = vx, vy
        self.mass = mass
        self.radius = BODY_RADIUS
        self.color = color

        # Draw body
        self.id = self.canvas.create_oval(
            self.x - self.radius, self.y - self.radius,
            self.x + self.radius, self.y + self.radius,
            fill=self.color
        )

        # Trail points
        self.trail = []

    def update_position(self):
        # Update body position
        self.canvas.coords(
            self.id,
            self.x - self.radius, self.y - self.radius,
            self.x + self.radius, self.y + self.radius
        )

        # Draw trail
        dot = self.canvas.create_oval(
            self.x, self.y, self.x+1, self.y+1,
            fill=self.color, outline=self.color
        )
        self.trail.append(dot)

        # Limit trail length
        if len(self.trail) > TRAIL_LENGTH:
            self.canvas.delete(self.trail.pop(0))

# ==========================
# Simulation Class
# ==========================
class NBodySim:
    def __init__(self, root):
        self.root = root
        self.root.title("3-Body Problem Simulator")

        self.canvas = tk.Canvas(root, width=800, height=600, bg="black")
        self.canvas.pack()

        # Create 3 bodies aimed at center
        self.bodies = self.create_bodies()

        self.running = False

        # Control panel
        control_frame = tk.Frame(root)
        control_frame.pack()

        tk.Button(control_frame, text="Start", command=self.start).pack(side="left")
        tk.Button(control_frame, text="Stop", command=self.stop).pack(side="left")
        tk.Button(control_frame, text="Reset", command=self.reset).pack(side="left")

        # Sliders for mass
        self.mass_sliders = []
        for i, body in enumerate(self.bodies):
            frame = tk.Frame(root)
            frame.pack()
            label = tk.Label(frame, text=f"Mass of Body {i+1} ({body.color})")
            label.pack(side="left")
            slider = tk.Scale(frame, from_=10, to=200, orient="horizontal",
                              length=200, command=lambda val, idx=i: self.update_mass(idx, val))
            slider.set(body.mass)
            slider.pack(side="left")
            self.mass_sliders.append(slider)

    def create_bodies(self):
        bodies = []
        colors = ["red", "blue", "green"]

        # Rough center of space
        center_x, center_y = 400, 300  

        for i in range(3):
            x = random.randint(200, 600)
            y = random.randint(150, 450)

            # Velocity pointing toward center
            dx = center_x - x
            dy = center_y - y
            dist = math.sqrt(dx*dx + dy*dy) + 1e-6
            vx = (dx / dist) * random.uniform(1, 2)
            vy = (dy / dist) * random.uniform(1, 2)

            # Add a small sideways component
            vx += random.uniform(-1, 1)
            vy += random.uniform(-1, 1)

            mass = random.randint(50, 120)
            bodies.append(Body(self.canvas, x, y, vx, vy, mass, colors[i]))
        return bodies

    def compute_forces(self):
        forces = [(0, 0) for _ in self.bodies]

        # compute pairwise forces
        for i, body1 in enumerate(self.bodies):
            for j, body2 in enumerate(self.bodies):
                if i == j:
                    continue
                dx = body2.x - body1.x
                dy = body2.y - body1.y
                dist = math.sqrt(dx*dx + dy*dy) + 1e-6  # avoid div 0
                f = G * body1.mass * body2.mass / (dist*dist)

                fx = f * dx / dist
                fy = f * dy / dist

                forces[i] = (forces[i][0] + fx, forces[i][1] + fy)

        return forces

    def update(self):
        if not self.running:
            return

        for _ in range(3):  # multiple physics steps per frame
            forces = self.compute_forces()
            for i, body in enumerate(self.bodies):
                fx, fy = forces[i]
                ax, ay = fx / body.mass, fy / body.mass
                body.vx += ax * DT
                body.vy += ay * DT
                body.x += body.vx * DT
                body.y += body.vy * DT
                body.update_position()

        self.root.after(20, self.update)

    def update_mass(self, index, value):
        """Update body mass when slider is moved"""
        self.bodies[index].mass = float(value)

    def start(self):
        if not self.running:
            self.running = True
            self.update()

    def stop(self):
        self.running = False

    def reset(self):
        self.stop()
        self.canvas.delete("all")
        self.bodies = self.create_bodies()
        # reset sliders to new body masses
        for i, body in enumerate(self.bodies):
            self.mass_sliders[i].set(body.mass)

# ==========================
# Run
# ==========================
if __name__ == "__main__":
    root = tk.Tk()
    app = NBodySim(root)
    root.mainloop()


## V2

In [11]:
import tkinter as tk
import math
import random
import time

# ==========================
# Physics & Sim constants
# ==========================
G = 0.5       # gravitational constant
DT = 0.2      # time step
SUBSTEPS = 3  # physics substeps per frame

TRAIL_LENGTH = 150
BODY_RADIUS = 12
W, H = 800, 600

# Collision threshold between bodies (treat them as soft spheres)
COLLIDE_DIST = BODY_RADIUS * 2.0 * 0.9  # a bit less than touching to be safe

# ==========================
# Body
# ==========================
class Body:
    def __init__(self, canvas, x, y, vx, vy, mass, color):
        self.canvas = canvas
        self.x, self.y = x, y
        self.vx, self.vy = vx, vy
        self.mass = mass
        self.radius = BODY_RADIUS
        self.color = color
        self.id = self.canvas.create_oval(
            self.x - self.radius, self.y - self.radius,
            self.x + self.radius, self.y + self.radius,
            fill=self.color
        )
        self.trail = []

    def update_draw(self):
        self.canvas.coords(
            self.id,
            self.x - self.radius, self.y - self.radius,
            self.x + self.radius, self.y + self.radius
        )
        dot = self.canvas.create_oval(
            self.x, self.y, self.x+1, self.y+1,
            fill=self.color, outline=self.color
        )
        self.trail.append(dot)
        if len(self.trail) > TRAIL_LENGTH:
            self.canvas.delete(self.trail.pop(0))

# ==========================
# Core physics helpers
# ==========================
def pairwise_forces(bodies):
    n = len(bodies)
    fx = [0.0]*n
    fy = [0.0]*n
    for i in range(n):
        for j in range(i+1, n):
            dx = bodies[j].x - bodies[i].x
            dy = bodies[j].y - bodies[i].y
            dist = math.hypot(dx, dy) + 1e-6
            fmag = G * bodies[i].mass * bodies[j].mass / (dist*dist)
            ux, uy = dx/dist, dy/dist
            fx_i, fy_i = fmag*ux, fmag*uy
            fx[i] += fx_i; fy[i] += fy_i
            fx[j] -= fx_i; fy[j] -= fy_i
    return fx, fy

def step_physics(bodies, width=W, height=H):
    fx, fy = pairwise_forces(bodies)
    # Integrate
    for i, b in enumerate(bodies):
        ax, ay = fx[i]/b.mass, fy[i]/b.mass
        b.vx += ax * DT
        b.vy += ay * DT
        b.x  += b.vx * DT
        b.y  += b.vy * DT

        # Reflective borders (perfectly elastic)
        if b.x - b.radius < 0:
            b.x = b.radius
            b.vx = -b.vx
        elif b.x + b.radius > width:
            b.x = width - b.radius
            b.vx = -b.vx

        if b.y - b.radius < 0:
            b.y = b.radius
            b.vy = -b.vy
        elif b.y + b.radius > height:
            b.y = height - b.radius
            b.vy = -b.vy

def any_collision(bodies):
    n = len(bodies)
    for i in range(n):
        for j in range(i+1, n):
            if math.hypot(bodies[i].x - bodies[j].x, bodies[i].y - bodies[j].y) < COLLIDE_DIST:
                return True
    return False

def kinetic_energy(bodies):
    return sum(0.5*b.mass*(b.vx*b.vx + b.vy*b.vy) for b in bodies)

# ==========================
# Tuning (Evolutionary Search)
# ==========================
def random_candidate():
    """Return (positions, velocities, masses) for 3 bodies."""
    params = []
    # Place around center, spaced
    for _ in range(3):
        x = random.uniform(0.25*W, 0.75*W)
        y = random.uniform(0.25*H, 0.75*H)
        # Directed toward center + small tangential term
        cx, cy = W/2, H/2
        dx, dy = cx - x, cy - y
        dist = math.hypot(dx, dy) + 1e-6
        ux, uy = dx/dist, dy/dist
        # speed magnitude
        base = random.uniform(0.8, 2.2)
        # tangential component
        tx, ty = -uy, ux
        tang = random.uniform(-0.8, 0.8)
        vx = base*ux + tang*tx
        vy = base*uy + tang*ty
        m = random.uniform(50, 120)
        params.append([x, y, vx, vy, m])
    return params

def mutate(params, scale=0.08):
    out = []
    for (x,y,vx,vy,m) in params:
        out.append([
            min(max(x + random.gauss(0, scale*W), BODY_RADIUS), W - BODY_RADIUS),
            min(max(y + random.gauss(0, scale*H), BODY_RADIUS), H - BODY_RADIUS),
            vx + random.gauss(0, scale*2.0),
            vy + random.gauss(0, scale*2.0),
            min(max(m + random.gauss(0, scale*150), 30), 200),
        ])
    return out

def candidate_to_bodies(canvas, params, colors=("red","blue","green")):
    bodies = []
    for p, c in zip(params, colors):
        x,y,vx,vy,m = p
        bodies.append(Body(canvas, x,y,vx,vy,m,c))
    return bodies

def score_candidate(params, steps=1200, substeps=1):
    """Run a short off-screen sim to score stability (higher is better)."""
    # Use a dummy list of lightweight Body objects (no canvas)
    class B: pass
    bodies = []
    for x,y,vx,vy,m in params:
        b = B()
        b.x, b.y, b.vx, b.vy, b.mass, b.radius = x,y,vx,vy,m,BODY_RADIUS
        bodies.append(b)
    survived = 0
    penalty = 0.0
    for t in range(steps):
        for _ in range(substeps):
            fx, fy = pairwise_forces(bodies)
            for i, b in enumerate(bodies):
                ax, ay = fx[i]/b.mass, fy[i]/b.mass
                b.vx += ax * DT
                b.vy += ay * DT
                b.x  += b.vx * DT
                b.y  += b.vy * DT

                # Reflective walls
                if b.x - b.radius < 0:
                    b.x = b.radius; b.vx = -b.vx
                elif b.x + b.radius > W:
                    b.x = W - b.radius; b.vx = -b.vx
                if b.y - b.radius < 0:
                    b.y = b.radius; b.vy = -b.vy
                elif b.y + b.radius > H:
                    b.y = H - b.radius; b.vy = -b.vy

        # collision ends run (strong instability)
        if any(math.hypot(bodies[i].x-bodies[j].x, bodies[i].y-bodies[j].y) < COLLIDE_DIST
               for i in range(3) for j in range(i+1,3)):
            break

        # Penalize extreme kinetic energy (chaotic ping-pong)
        ke = sum(0.5*b.mass*(b.vx*b.vx + b.vy*b.vy) for b in bodies)
        penalty += 0.0005 * ke
        survived += 1

    # Score = survived time - penalties; small bonus for spread (less clustering)
    spread = 0.0
    for i in range(3):
        for j in range(i+1,3):
            spread += math.hypot(bodies[i].x - bodies[j].x, bodies[i].y - bodies[j].y)
    spread /= 3.0
    return survived + 0.01*spread - penalty

def evolutionary_search(generations=15, pop_size=28, elite_k=4):
    # Initialize population
    pop = [random_candidate() for _ in range(pop_size)]
    scores = [score_candidate(p, steps=900, substeps=1) for p in pop]
    best_params = pop[max(range(pop_size), key=lambda i: scores[i])]
    best_score = max(scores)

    for _ in range(generations):
        # Select elites
        ranked = sorted(range(pop_size), key=lambda i: scores[i], reverse=True)
        elites = [pop[i] for i in ranked[:elite_k]]

        # Refill population via mutated elites + fresh randoms
        new_pop = elites[:]
        while len(new_pop) < pop_size:
            parent = random.choice(elites)
            child = mutate(parent, scale=random.uniform(0.04, 0.12))
            new_pop.append(child)
        pop = new_pop
        scores = [score_candidate(p, steps=1000, substeps=1) for p in pop]
        gen_best_idx = max(range(pop_size), key=lambda i: scores[i])
        if scores[gen_best_idx] > best_score:
            best_score = scores[gen_best_idx]
            best_params = pop[gen_best_idx]

    return best_params, best_score

# ==========================
# GUI App
# ==========================
class NBodySim:
    def __init__(self, root):
        self.root = root
        self.root.title("3-Body Problem Simulator (with Borders + Auto-Tune)")
        self.canvas = tk.Canvas(root, width=W, height=H, bg="black")
        self.canvas.pack()

        self.bodies = self._create_bodies_directed()
        self.running = False

        # Controls
        controls = tk.Frame(root); controls.pack(pady=4)
        tk.Button(controls, text="Start", command=self.start).pack(side="left", padx=2)
        tk.Button(controls, text="Stop",  command=self.stop).pack(side="left", padx=2)
        tk.Button(controls, text="Reset", command=self.reset).pack(side="left", padx=2)
        tk.Button(controls, text="Auto-Tune (Find Stable)", command=self.autotune).pack(side="left", padx=8)

        # Status
        self.status = tk.Label(root, text="Ready", fg="white", bg="#333")
        self.status.pack(fill="x")

        # Mass sliders
        self.mass_sliders = []
        for i, body in enumerate(self.bodies):
            f = tk.Frame(root); f.pack()
            tk.Label(f, text=f"Mass of Body {i+1} ({body.color})").pack(side="left")
            s = tk.Scale(f, from_=30, to=200, orient="horizontal", length=220,
                         command=lambda val, idx=i: self.update_mass(idx, val))
            s.set(body.mass); s.pack(side="left")
            self.mass_sliders.append(s)

    def _create_bodies_directed(self):
        colors = ["red","blue","green"]
        cx, cy = W/2, H/2
        bodies = []
        for c in colors:
            x = random.uniform(0.3*W, 0.7*W)
            y = random.uniform(0.3*H, 0.7*H)
            dx, dy = cx - x, cy - y
            dist = math.hypot(dx, dy) + 1e-6
            ux, uy = dx/dist, dy/dist
            base = random.uniform(1.0, 2.0)
            tx, ty = -uy, ux
            tang = random.uniform(-0.8, 0.8)
            vx = base*ux + tang*tx
            vy = base*uy + tang*ty
            m  = random.uniform(50, 120)
            bodies.append(Body(self.canvas, x,y,vx,vy,m,c))
        return bodies

    def _from_params(self, params):
        self.canvas.delete("all")
        colors = ["red","blue","green"]
        self.bodies = []
        for p, c in zip(params, colors):
            x,y,vx,vy,m = p
            self.bodies.append(Body(self.canvas, x,y,vx,vy,m,c))
        for i, b in enumerate(self.bodies):
            self.mass_sliders[i].set(b.mass)

    def update_mass(self, idx, val):
        self.bodies[idx].mass = float(val)

    def compute_and_update(self):
        if not self.running: return
        for _ in range(SUBSTEPS):
            step_physics(self.bodies, W, H)
        for b in self.bodies:
            b.update_draw()
        self.root.after(16, self.compute_and_update)

    def start(self):
        if not self.running:
            self.running = True
            self.compute_and_update()
            self.status.config(text="Running")

    def stop(self):
        self.running = False
        self.status.config(text="Stopped")

    def reset(self):
        self.stop()
        self.canvas.delete("all")
        self.bodies = self._create_bodies_directed()
        for i, b in enumerate(self.bodies):
            self.mass_sliders[i].set(b.mass)
        self.status.config(text="Reset complete")

    def autotune(self):
        # Pause sim during search
        was_running = self.running
        self.stop()
        self.status.config(text="Auto-tuning… this will run a quick search locally.")

        # Run evolutionary search (blocking). Keep values modest so it finishes quickly.
        t0 = time.time()
        params, best = evolutionary_search(generations=12, pop_size=26, elite_k=4)
        t1 = time.time()

        # Load best config
        self._from_params(params)
        self.status.config(text=f"Auto-tune done. Score {best:.1f}. Took {t1 - t0:.1f}s.")
        if was_running:
            self.start()

# ==========================
# Run
# ==========================
if __name__ == "__main__":
    root = tk.Tk()
    app = NBodySim(root)
    root.mainloop()


## V3

In [15]:
#!/usr/bin/env python3
"""
N-body 3-body simulator with GUI, presets (equilateral, figure-eight, binary+satellite),
elastic borders, trails, mass sliders, and a slow evolutionary ML search for stable/periodic configs.
Saves checkpoints to `ml_checkpoint.pkl`.
"""
import tkinter as tk
from tkinter import ttk, messagebox
import math, random, time, pickle, os
import threading

# -------------------------
# Simulation / display params
# -------------------------
W, H = 800, 600
BODY_RADIUS = 12               # fixed visual radius
TRAIL_LENGTH = 200
G = 0.5                        # gravitational constant used in sim (visual tuning)
DT = 0.5                      # timestep
SUBSTEPS = 2                   # physics substeps per frame
COLLIDE_DIST = BODY_RADIUS*2*0.95

# Checkpoint filename
CHECKPOINT_FILE = "ml_checkpoint.pkl"

# -------------------------
# Body class
# -------------------------
class Body:
    def __init__(self, canvas, x, y, vx, vy, mass, color):
        self.canvas = canvas
        self.x, self.y = float(x), float(y)
        self.vx, self.vy = float(vx), float(vy)
        self.mass = float(mass)
        self.radius = BODY_RADIUS
        self.color = color
        self.id = self.canvas.create_oval(
            self.x - self.radius, self.y - self.radius,
            self.x + self.radius, self.y + self.radius,
            fill=self.color, outline=self.color
        )
        self.trail = []

    def update_draw(self):
        # update body circle
        self.canvas.coords(
            self.id,
            self.x - self.radius, self.y - self.radius,
            self.x + self.radius, self.y + self.radius
        )
        # small dot for trail
        dot = self.canvas.create_oval(self.x, self.y, self.x+1, self.y+1, fill=self.color, outline=self.color)
        self.trail.append(dot)
        if len(self.trail) > TRAIL_LENGTH:
            oldest = self.trail.pop(0)
            try:
                self.canvas.delete(oldest)
            except:
                pass

# -------------------------
# Physics helpers
# -------------------------
def compute_pairwise_forces(bodies):
    n = len(bodies)
    fx = [0.0]*n
    fy = [0.0]*n
    for i in range(n):
        bi = bodies[i]
        for j in range(i+1, n):
            bj = bodies[j]
            dx = bj.x - bi.x
            dy = bj.y - bi.y
            dist = math.hypot(dx, dy) + 1e-6
            # inverse-square law
            f = G * bi.mass * bj.mass / (dist*dist)
            ux, uy = dx/dist, dy/dist
            fx_i = f * ux
            fy_i = f * uy
            fx[i] += fx_i; fy[i] += fy_i
            fx[j] -= fx_i; fy[j] -= fy_i
    return fx, fy

def step_physics(bodies, width=W, height=H):
    fx, fy = compute_pairwise_forces(bodies)
    for i, b in enumerate(bodies):
        ax = fx[i] / b.mass
        ay = fy[i] / b.mass
        b.vx += ax * DT
        b.vy += ay * DT
        b.x  += b.vx * DT
        b.y  += b.vy * DT

        # elastic borders
        if b.x - b.radius < 0:
            b.x = b.radius
            b.vx = -b.vx
        elif b.x + b.radius > width:
            b.x = width - b.radius
            b.vx = -b.vx
        if b.y - b.radius < 0:
            b.y = b.radius
            b.vy = -b.vy
        elif b.y + b.radius > height:
            b.y = height - b.radius
            b.vy = -b.vy

def any_collision_state(bodies):
    n = len(bodies)
    for i in range(n):
        for j in range(i+1, n):
            if math.hypot(bodies[i].x - bodies[j].x, bodies[i].y - bodies[j].y) < COLLIDE_DIST:
                return True
    return False

# -------------------------
# Presets: Equilateral, Figure-eight (approx), Binary+satellite
# -------------------------
def preset_equilateral(center_x=W/2, center_y=H/2, radius=160, mass=80):
    # Three equal masses on circle, tangential velocities chosen to roughly balance centripetal vs gravity.
    bodies = []
    m = mass
    for k, color in enumerate(("red","blue","green")):
        angle = 2*math.pi*k/3
        x = center_x + radius*math.cos(angle)
        y = center_y + radius*math.sin(angle)
        # tangential direction
        tx = -math.sin(angle)
        ty = math.cos(angle)
        # a heuristic for tangential speed to get circular motion:
        # v ~ sqrt(G * (sum_masses) / radius)
        v = math.sqrt(G * (3*m) / (radius+1e-6)) * 0.75
        vx, vy = v*tx, v*ty
        bodies.append([x,y,vx,vy,m,color])
    return bodies

def preset_figure_eight(center_x=W/2, center_y=H/2, scale=150):
    # Known normalized initial conditions for the figure-eight (Moore/Chenciner)
    # Positions/velocities taken from literature (unit system). We'll scale and center them.
    # Source ICs (approx): positions {(0.97000436, -0.24308753), (-0.97000436, 0.24308753), (0,0)}
    # velocities {(0.4662036850, 0.43236573), (0.4662036850, 0.43236573), (-0.93240737, -0.86473146)}
    pts = [
        (0.97000436, -0.24308753,  0.4662036850, 0.43236573),
        (-0.97000436, 0.24308753,  0.4662036850, 0.43236573),
        (0.0, 0.0, -0.93240737, -0.86473146)
    ]
    bodies = []
    mass = 60.0
    for (px,py,vx,vy), color in zip(pts, ("red","blue","green")):
        x = center_x + px*scale
        y = center_y + py*scale
        # scale velocities down to canvas units
        vx_s = vx * (scale/120.0)
        vy_s = vy * (scale/120.0)
        bodies.append([x,y,vx_s,vy_s,mass,color])
    return bodies

def preset_binary_satellite(center_x=W/2, center_y=H/2, sep=90):
    # Two heavy masses orbiting each other, third small satellite around them
    m1 = 120.0
    m2 = 100.0
    m3 = 35.0
    x1, y1 = center_x - sep/2, center_y
    x2, y2 = center_x + sep/2, center_y
    # velocities for circular orbit around barycenter:
    mu = G*(m1 + m2)
    # approx orbital speed for each around barycenter r ~ sep/2
    v1 = math.sqrt(G*m2 / (sep + 1e-6)) * 0.9
    v2 = math.sqrt(G*m1 / (sep + 1e-6)) * 0.9
    # directions: perpendicular to separation
    v1x, v1y = 0, -v1
    v2x, v2y = 0, v2
    # satellite: placed further out
    x3, y3 = center_x, center_y - 2.5*sep
    # velocity tangent for orbit
    v3 = math.sqrt(G*(m1+m2) / (2.5*sep + 1e-6)) * 0.8
    v3x, v3y = v3, 0
    bodies = [
        [x1,y1,v1x,v1y,m1,"red"],
        [x2,y2,v2x,v2y,m2,"blue"],
        [x3,y3,v3x,v3y,m3,"green"]
    ]
    return bodies

# -------------------------
# Evolutionary / ML-style search (slow & thorough)
# -------------------------
def random_candidate():
    params = []
    for _ in range(3):
        x = random.uniform(0.2*W, 0.8*W)
        y = random.uniform(0.2*H, 0.8*H)
        # direction toward center + small tangential
        cx, cy = W/2, H/2
        dx, dy = cx - x, cy - y
        dist = math.hypot(dx, dy) + 1e-6
        ux, uy = dx/dist, dy/dist
        base = random.uniform(0.8, 2.4)
        tx, ty = -uy, ux
        tang = random.uniform(-1.0, 1.0)
        vx = base*ux + tang*tx
        vy = base*uy + tang*ty
        m = random.uniform(35, 150)
        params.append([x,y,vx,vy,m])
    return params

def mutate(params, scale=0.08):
    out = []
    for (x,y,vx,vy,m) in params:
        out.append([
            min(max(x + random.gauss(0, scale*W), BODY_RADIUS), W - BODY_RADIUS),
            min(max(y + random.gauss(0, scale*H), BODY_RADIUS), H - BODY_RADIUS),
            vx + random.gauss(0, scale*2.0),
            vy + random.gauss(0, scale*2.0),
            min(max(m + random.gauss(0, scale*150), 20.0), 220.0),
        ])
    return out

def _score_candidate(params, steps=1500, sample_period=300):
    # simulate in-memory (no canvas) and compute:
    # - survival time (no collisions)
    # - small penalty for high energy
    # - periodicity metric: sample snapshots every sample_period and measure how similar they are
    class B:
        pass
    bodies = []
    for x,y,vx,vy,m in params:
        b = B()
        b.x, b.y, b.vx, b.vy, b.mass, b.radius = x, y, vx, vy, m, BODY_RADIUS
        bodies.append(b)

    survived = 0
    penalty = 0.0
    snapshots = []
    for t in range(steps):
        # physics substeps
        fx, fy = compute_pairwise_forces(bodies)
        for i, b in enumerate(bodies):
            ax, ay = fx[i]/b.mass, fy[i]/b.mass
            b.vx += ax * DT
            b.vy += ay * DT
            b.x += b.vx * DT
            b.y += b.vy * DT
            # elastic walls
            if b.x - b.radius < 0:
                b.x = b.radius; b.vx = -b.vx
            elif b.x + b.radius > W:
                b.x = W - b.radius; b.vx = -b.vx
            if b.y - b.radius < 0:
                b.y = b.radius; b.vy = -b.vy
            elif b.y + b.radius > H:
                b.y = H - b.radius; b.vy = -b.vy

        # collision check
        collided = False
        for i in range(3):
            for j in range(i+1, 3):
                if math.hypot(bodies[i].x - bodies[j].x, bodies[i].y - bodies[j].y) < COLLIDE_DIST:
                    collided = True
                    break
            if collided: break
        if collided:
            break

        # penalty: high kinetic energy (chaotic ping-pong)
        ke = sum(0.5*b.mass*(b.vx*b.vx + b.vy*b.vy) for b in bodies)
        penalty += 0.0006 * ke
        survived += 1

        # collect snapshots for periodicity
        if t % sample_period == 0:
            snapshots.append([(b.x, b.y) for b in bodies])

    # periodicity score: compute pairwise RMS between snapshots (low RMS -> repeats)
    periodicity_score = 0.0
    if len(snapshots) >= 2:
        # compute average RMS between successive snapshots (lower is better)
        rms_vals = []
        for i in range(1, len(snapshots)):
            s0 = snapshots[0]
            s1 = snapshots[i]
            # find minimal matching among permutations (since labels may permute); brute force 3! = 6 ok
            best = float("inf")
            import itertools
            for perm in itertools.permutations(range(3)):
                s = 0.0
                for a in range(3):
                    x0,y0 = s0[a]
                    x1,y1 = s1[perm[a]]
                    s += (x0-x1)**2 + (y0-y1)**2
                rms = math.sqrt(s/3.0)
                if rms < best: best = rms
            rms_vals.append(best)
        avg_rms = sum(rms_vals)/len(rms_vals)
        # convert to score: small avg_rms -> higher periodicity_score
        periodicity_score = max(0.0, 50.0 - avg_rms*0.05)  # tuned heuristically

    # spread bonus (favor well separated)
    spread = 0.0
    for i in range(3):
        for j in range(i+1, 3):
            spread += math.hypot(bodies[i].x - bodies[j].x, bodies[i].y - bodies[j].y)
    spread /= 3.0

    score = survived + 0.01*spread - penalty + periodicity_score
    return score

def evolutionary_search_ui(update_callback=None, generations=50, pop_size=40, elite_k=6, resume=True):
    """
    A somewhat heavy, thorough evolutionary search.
    update_callback(iteration, best_score, progress_fraction) -> should be called to update UI.
    Returns best_params, best_score.
    """
    # try to load checkpoint
    best_from_checkpoint = None
    if resume and os.path.exists(CHECKPOINT_FILE):
        try:
            with open(CHECKPOINT_FILE, "rb") as f:
                ck = pickle.load(f)
                best_from_checkpoint = ck.get("best_params", None)
                best_score_ck = ck.get("best_score", None)
        except Exception as e:
            best_from_checkpoint = None

    # initialize population
    pop = [random_candidate() for _ in range(pop_size)]
    if best_from_checkpoint:
        # inject into population
        pop[0] = best_from_checkpoint

    scores = [ _score_candidate(p, steps=1400, sample_period=350) for p in pop ]
    best_idx = max(range(pop_size), key=lambda i: scores[i])
    best_params = pop[best_idx]
    best_score = scores[best_idx]

    # main loop
    total_iters = generations
    for gen in range(generations):
        # rank
        ranked = sorted(range(pop_size), key=lambda i: scores[i], reverse=True)
        elites = [pop[i] for i in ranked[:elite_k]]

        # produce new population
        new_pop = elites[:]
        while len(new_pop) < pop_size:
            parent = random.choice(elites)
            child = mutate(parent, scale=random.uniform(0.04, 0.12))
            new_pop.append(child)
        pop = new_pop
        # evaluate
        scores = [ _score_candidate(p, steps=1600, sample_period=400) for p in pop ]
        gen_best_idx = max(range(pop_size), key=lambda i: scores[i])
        gen_best_score = scores[gen_best_idx]
        if gen_best_score > best_score:
            best_score = gen_best_score
            best_params = pop[gen_best_idx]
            # save checkpoint
            try:
                with open(CHECKPOINT_FILE, "wb") as f:
                    pickle.dump({"best_params": best_params, "best_score": best_score, "time": time.time()}, f)
            except Exception:
                pass

        # UI update
        if update_callback:
            update_callback(gen+1, best_score, (gen+1)/total_iters)

    # final checkpoint
    try:
        with open(CHECKPOINT_FILE, "wb") as f:
            pickle.dump({"best_params": best_params, "best_score": best_score, "time": time.time()}, f)
    except Exception:
        pass

    return best_params, best_score

# -------------------------
# GUI App
# -------------------------
class NBodyApp:
    def __init__(self, root):
        self.root = root
        self.root.title("3-Body Simulator — Presets + ML Search")
        # main canvas
        self.canvas = tk.Canvas(root, width=W, height=H, bg="black")
        self.canvas.pack()

        # bodies list
        self.bodies = []
        # create default random directed bodies
        self.load_presets("Manual")  # starts with a manual random set

        # controls
        ctrl = tk.Frame(root)
        ctrl.pack(fill="x", pady=4)

        # mode dropdown
        tk.Label(ctrl, text="Simulation Mode:").pack(side="left", padx=4)
        self.mode_var = tk.StringVar(value="Manual")
        modes = ["Manual", "Equilateral Triangle", "Figure-Eight", "Binary+Satellite", "ML Search"]
        self.mode_menu = ttk.Combobox(ctrl, values=modes, state="readonly", width=18, textvariable=self.mode_var)
        self.mode_menu.pack(side="left", padx=4)

        tk.Button(ctrl, text="Apply Preset", command=self.apply_preset).pack(side="left", padx=4)
        tk.Button(ctrl, text="Start", command=self.start_sim).pack(side="left", padx=4)
        tk.Button(ctrl, text="Stop", command=self.stop_sim).pack(side="left", padx=4)
        tk.Button(ctrl, text="Reset", command=self.reset_sim).pack(side="left", padx=4)

        # ML Search button
        self.ml_btn = tk.Button(ctrl, text="Run ML Search (Slow)", command=self.run_ml_search)
        self.ml_btn.pack(side="left", padx=8)

        # status & progress
        status_frame = tk.Frame(root)
        status_frame.pack(fill="x")
        self.status_label = tk.Label(status_frame, text="Ready", anchor="w")
        self.status_label.pack(side="left", fill="x", expand=True, padx=6)
        self.progress = ttk.Progressbar(status_frame, orient="horizontal", length=220, mode="determinate")
        self.progress.pack(side="right", padx=6)

        # mass sliders
        self.mass_sliders = []
        for i in range(3):
            f = tk.Frame(root)
            f.pack(anchor="w", padx=6)
            lbl = tk.Label(f, text=f"Mass Body {i+1}", width=12)
            lbl.pack(side="left")
            s = tk.Scale(f, from_=20, to=220, orient="horizontal", length=260,
                         command=lambda val, idx=i: self.update_mass(idx, val))
            s.set(80)
            s.pack(side="left")
            self.mass_sliders.append(s)

        # simulation loop control
        self.running = False
        self._anim_after_id = None

        # lock for thread-safety with ML search thread
        self.lock = threading.Lock()

    # -----------
    # load / spawn
    # -----------
    def clear_canvas(self):
        self.canvas.delete("all")

    def spawn_bodies_from_params(self, params):
        # params: list of [x,y,vx,vy,m,color]
        self.clear_canvas()
        self.bodies = []
        colors = ["red","blue","green"]
        for i, p in enumerate(params):
            x,y,vx,vy,m,color = p
            self.bodies.append(Body(self.canvas, x,y,vx,vy,m,color or colors[i]))
        # update sliders
        for i, b in enumerate(self.bodies):
            try:
                self.mass_sliders[i].set(int(b.mass))
            except:
                pass

    def load_presets(self, mode):
        if mode == "Manual":
            # directed random initial config
            params = []
            cx, cy = W/2, H/2
            for col in ("red","blue","green"):
                x = random.uniform(0.25*W, 0.75*W)
                y = random.uniform(0.25*H, 0.75*H)
                dx, dy = cx - x, cy - y
                dist = math.hypot(dx, dy) + 1e-6
                ux, uy = dx/dist, dy/dist
                base = random.uniform(1.0, 2.0)
                tx, ty = -uy, ux
                tang = random.uniform(-1.0, 1.0)
                vx = base*ux + tang*tx
                vy = base*uy + tang*ty
                m = random.uniform(50, 120)
                params.append([x,y,vx,vy,m,col])
            self.spawn_bodies_from_params(params)
        elif mode == "Equilateral Triangle":
            params = preset_equilateral()
            self.spawn_bodies_from_params(params)
        elif mode == "Figure-Eight":
            params = preset_figure_eight()
            self.spawn_bodies_from_params(params)
        elif mode == "Binary+Satellite":
            params = preset_binary_satellite()
            self.spawn_bodies_from_params(params)
        elif mode == "ML Search":
            # load checkpoint if exists and show it (but do not run)
            if os.path.exists(CHECKPOINT_FILE):
                try:
                    with open(CHECKPOINT_FILE, "rb") as f:
                        ck = pickle.load(f)
                        params = []
                        bp = ck.get("best_params")
                        if bp:
                            # checkpoint stores list of [x,y,vx,vy,m] triples per body
                            colors = ["red","blue","green"]
                            for (x,y,vx,vy,m), c in zip(bp, colors):
                                params.append([x,y,vx,vy,m,c])
                            self.spawn_bodies_from_params(params)
                            self.status_label.config(text=f"Loaded checkpoint (score={ck.get('best_score'):.1f})")
                            return
                except Exception:
                    pass
            # else spawn random
            self.load_presets("Manual")

    # -----------
    # GUI actions
    # -----------
    def apply_preset(self):
        mode = self.mode_var.get()
        if mode == "Manual":
            self.load_presets("Manual")
        elif mode == "Equilateral Triangle":
            self.load_presets("Equilateral Triangle")
        elif mode == "Figure-Eight":
            self.load_presets("Figure-Eight")
        elif mode == "Binary+Satellite":
            self.load_presets("Binary+Satellite")
        elif mode == "ML Search":
            # show checkpoint if available
            self.load_presets("ML Search")
            messagebox.showinfo("ML Search", "To run ML Search, click 'Run ML Search (Slow)'. This search is slow and thorough.")
        self.status_label.config(text=f"Preset '{mode}' applied.")

    def update_mass(self, idx, val):
        try:
            self.bodies[idx].mass = float(val)
        except Exception:
            pass

    def start_sim(self):
        if not self.running:
            self.running = True
            self.status_label.config(text="Running simulation")
            self._simulate_step()

    def stop_sim(self):
        if self.running:
            self.running = False
            self.status_label.config(text="Stopped")

    def reset_sim(self):
        # reload current mode preset
        self.stop_sim()
        self.apply_preset()
        self.status_label.config(text="Reset done")

    # ----------
    # simulation loop
    # ----------
    def _simulate_step(self):
        if not self.running:
            return
        # several physics substeps per frame for stability
        for _ in range(SUBSTEPS):
            step_physics(self.bodies, W, H)
        # draw
        for b in self.bodies:
            b.update_draw()
        # schedule next
        self._anim_after_id = self.root.after(20, self._simulate_step)

    # ----------
    # ML search handling (threaded)
    # ----------
    def run_ml_search(self):
        if self.mode_var.get() != "ML Search":
            messagebox.showwarning("Mode", "Set mode to 'ML Search' first (dropdown) then click Run ML Search.")
            return
        # disable ML button while running
        self.ml_btn.config(state="disabled")
        self.status_label.config(text="ML search starting... (this may take a while)")
        self.progress['value'] = 0
        # run in background thread
        t = threading.Thread(target=self._ml_search_thread, daemon=True)
        t.start()
        # start UI poll for progress
        self.root.after(200, self._poll_ml_progress)

    def _ml_search_thread(self):
        # callback from evolutionary search to update status
        def update_callback(iteration, best_score, frac):
            # store to attributes in thread-safe way
            with self.lock:
                self._ml_progress_frac = frac
                self._ml_best_score = best_score
                self._ml_iteration = iteration
        # run long evolutionary search (parameters chosen to emphasize thoroughness)
        gens = 65            # increase for even more thorough (slow)
        pop = 60
        elite = 8
        try:
            best_params, best_score = evolutionary_search_ui(
                update_callback=update_callback,
                generations=gens,
                pop_size=pop,
                elite_k=elite,
                resume=True
            )
            # convert best_params to displayable bodies and spawn on canvas in UI thread
            colors = ["red","blue","green"]
            params_with_color = []
            for (x,y,vx,vy,m), c in zip(best_params, colors):
                params_with_color.append([x,y,vx,vy,m,c])
            # set attributes for main thread to pick up
            with self.lock:
                self._ml_done = True
                self._ml_result = (params_with_color, best_score)
        except Exception as e:
            with self.lock:
                self._ml_error = str(e)
        finally:
            with self.lock:
                self._ml_finished_flag = True

    def _poll_ml_progress(self):
        with self.lock:
            frac = getattr(self, "_ml_progress_frac", 0.0)
            best_score = getattr(self, "_ml_best_score", None)
            iter_no = getattr(self, "_ml_iteration", None)
            finished = getattr(self, "_ml_finished_flag", False)
            done = getattr(self, "_ml_done", False)
            result = getattr(self, "_ml_result", None)
            error = getattr(self, "_ml_error", None)
        # update UI elements
        self.progress['value'] = frac * 100.0
        if best_score is not None:
            self.status_label.config(text=f"ML search: gen {iter_no} — best score {best_score:.2f}")
        if finished:
            self.ml_btn.config(state="normal")
            self.progress['value'] = 100.0
            if error:
                self.status_label.config(text=f"ML search error: {error}")
                messagebox.showerror("ML Error", f"ML search encountered an error:\n{error}")
            elif done and result:
                params_with_color, best_score = result
                self.spawn_bodies_from_params(params_with_color)
                self.status_label.config(text=f"ML done. Best score: {best_score:.2f}. Loaded configuration.")
                # optionally auto-start the sim
                # self.start_sim()
            else:
                self.status_label.config(text="ML finished but no result loaded.")
            # cleanup flags
            with self.lock:
                self._ml_finished_flag = False
                self._ml_done = False
                self._ml_result = None
                self._ml_progress_frac = 0.0
                self._ml_best_score = None
                self._ml_iteration = None
                self._ml_error = None
            return
        # continue polling
        self.root.after(250, self._poll_ml_progress)

# -------------------------
# Run Application
# -------------------------
def main():
    root = tk.Tk()
    app = NBodyApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
